<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/EqBalancing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
rxn=r'aKNO3 + 5C12H22O11 -> bN2 + cCO2 + dH2O + eK2CO3'

In [3]:
rxn = re.sub(r'\s+','',rxn)
left_rxn,right_rxn = rxn.split('->')

In [4]:
stoic_pat=re.compile(r'(.*?)([A-Z].*)')
atom_pat=re.compile(r'([A-Z][a-z]?)(\d*)')

In [5]:
def process_side(side,sign):
    stoic_mols = side.split('+')
    mol_dict={}
    for stoic_mol in stoic_mols:
        res = stoic_pat.match(stoic_mol)
        stoic=res[1]
        stoic = 1 if stoic=='' else stoic
        mol=res[2]
        atom_nums = atom_pat.findall(mol)
        d={'stoic':stoic}
        for atom,num in atom_nums:
            d[atom]=sign*(1 if num=='' else int(num))
        mol_dict[mol]=d
    df=pd.DataFrame(mol_dict)
    df=df.fillna(0)
    return df

In [6]:
df_left=process_side(left_rxn,-1)
df_right=process_side(right_rxn,1)
df=pd.concat((df_left,df_right),axis=1)
fixed_col=pd.to_numeric(df.loc['stoic',:],errors='coerce').notna()

atom_num=df.iloc[1:,:].astype(int)
fixed_atom_num=atom_num.loc[:,fixed_col].to_numpy()
fixed_stoic=df.loc['stoic',fixed_col].astype(int).to_numpy()

In [7]:
rhs=-np.sum(fixed_stoic*fixed_atom_num,axis=1)
A=atom_num.loc[:,~fixed_col].to_numpy()

In [8]:
unk_stoic=np.linalg.solve(A,rhs)
df_stoic=df.loc['stoic',:]
df_stoic[~fixed_col]=unk_stoic

In [9]:
nleft=len(left_rxn.split('+'))
nright=len(right_rxn.split('+'))

In [10]:
for i,(stoic, mol) in enumerate(zip(df_stoic[:nleft],df.columns[:nleft])):
    print(f'{int(stoic)}{mol}',end='')
    if i<nleft-1:
        print(' + ',end='')
print(' -> ', end='')
for i,(stoic, mol) in enumerate(zip(df_stoic[nleft:],df.columns[nleft:])):
    print(f'{int(stoic)}{mol}',end='')
    if i<nright-1:
        print(' + ',end='')

48KNO3 + 5C12H22O11 -> 24N2 + 36CO2 + 55H2O + 24K2CO3